**import**

In [ ]:
from path_finder import GreedySearch
from path_finder import BFS
from path_finder import DFS
from path_finder import AStarSearch
from graph_generator import read_input_file
from matplotlib import pyplot as plt
import time
from IPython.display import display

%matplotlib notebook

**test function**

In [ ]:
import timeit
def test(algorithm, start, goal, matrix, is_print=True):
    path_finder = algorithm(start, goal)
    PATH_VALUE = 9

    start_tick = timeit.default_timer()
    path = path_finder.search(matrix)
    end_tick = timeit.default_timer()
    matrix[start] = PATH_VALUE + 2
    matrix[goal] = PATH_VALUE + 2
    fig = plt.figure()
    fig.canvas.draw()
    
    if path:
        print("Found path")
        for pos in path:
            matrix[pos] = PATH_VALUE
            fig.canvas.draw()
            plt.imshow(matrix, cmap="tab20c")
            plt.pause(0.5)
            plt.show()
    else:
        print("Not found path")
    
    if is_print:
        print(f'distance: {len(path)}, time: {end_tick - start_tick}')
        
        fig.canvas.draw()
        plt.imshow(matrix, cmap="tab20c")
        plt.pause(0.5)
        plt.show()

**main**

In [ ]:
# from path_finder import BFS
# from graph_generator import read_input_file

def read_input_and_test(algorithm):
    input_path = 'sample_input.txt'
    matrix, start, goal = read_input_file(input_path)
    if algorithm != None:
        test(algorithm, start, goal, matrix)

if __name__ == '__main__':
#     input_path = 'sample_input.txt'
#     matrix, start, goal = read_input_file(input_path)
    print('BFS')
    read_input_and_test(BFS)
    print('DFS')
    read_input_and_test(DFS)
    print('A')
    read_input_and_test(GreedySearch)
    print('A*')
    read_input_and_test(AStarSearch)

# level 3:

## Collecting all points before reaching the goal

In [5]:
def pretty(d, indent=0):
   for key, value in d.items():
      print('\t' * indent + str(key))
      if isinstance(value, dict):
         pretty(value, indent+1)
      else:
         print('\t' * (indent+1) + str(value))

In [6]:
from graph_generator import GraphGenerator

def read_multi_point_input(fpath:str, verbose:bool=False, fill:bool=False):
    """ Read input file with TA Format
    Input:
        fpath: Path to input txt file
        verbose: Print to screen to debug
        fill: Should we fill the polygon
    Output:
        graph: (Numpy array) Matrix to represent graph value
        start: (Tuple of int) Start point
        goal: (Tuple of int) Goal point
    """
    f = open(fpath)
    l = f.readline()
    w, h = [int(x) for x in l.strip().split(',')]

    if verbose:
        print("Read from file....")
        print(f"Width={w}, Height={h}")

    generator = GraphGenerator((h, w), fill=fill)
    
    l = f.readline()
    
    #xStart, yStart, xGoal, yGoal = [int(x) for x in l.strip().split(',')]
    list_coor = l.strip().split(',')
    list_x = list_coor[1::2]
    list_y = list_coor[::2]
    list_x = [int(x) for x in list_x]
    list_y = [int(y) for y in list_y]

    xStart, yStart, xGoal, yGoal = list_x[0], list_y[0], list_x[1], list_y[1]
    list_point = [(list_x[i], list_y[i]) for i in range(2, len(list_x))]
    
    
    if verbose:
        print(f"Start: ({xStart}, {yStart})")
        print(f"Goal: ({xGoal}, {yGoal})")

    num_polygon = int(f.readline().strip())

    if verbose:
        print(f"Found {num_polygon} Polygons")
    for pInd in range(num_polygon):
        l = f.readline()
        point_list = [int(x) for x in l.strip().split(',')]
        polygon = list(zip(point_list[::2], point_list[1::2]))

        generator.add_polygon(polygon)

    f.close()

    if verbose:
        print("Done load input")
        generator.plot_graph()

    return generator.generate_graph(), (xStart, yStart), (xGoal, yGoal), list_point


## main - all points search

In [15]:
file_path = "multi_points.txt"
#file_path = "sample_input.txt"
matrix, start, goal, list_point = read_multi_point_input(file_path)
#matrix, start, goal = read_input_file(file_path)

In [8]:
print(list_point)
print(start)
print(goal)

[(5, 1), (2, 8), (9, 8), (19, 3), (2, 20)]
(1, 1)
(19, 20)


**Use_heapq = True -- greedy**

In [11]:
from path_finder import AllPointSearch
all_point_finder = AllPointSearch(start, goal, list_point, use_heapq=True, algo_to_find_shortest_point=BFS)

In [12]:
total_dist, final_path = all_point_finder.search_always_choose_nearest_point(matrix)

**Use_heapq = False -- brute force**

In [16]:
from path_finder import AllPointSearch
all_point_finder = AllPointSearch(start, goal, list_point, use_heapq=False, algo_to_find_shortest_point=BFS)

In [17]:
total_dist, final_path = all_point_finder.search_shortest_with_all_possible_way(matrix)

(1, 1)
(5, 1)
(19, 3)
(9, 8)
(2, 8)
(2, 20)


In [ ]:
fig = plt.figure()
fig.canvas.draw()
PATH_VALUE = 9

matrix[start] = 7
matrix[goal] = 7
for p in list_point:
    matrix[p] = 8
    
if final_path:
    print("Found path")
    for pos in final_path:
        if matrix[pos] != PATH_VALUE:
            matrix[pos] = PATH_VALUE
        plt.imshow(matrix, cmap="tab20c")
        fig.canvas.draw()
        plt.pause(0.5)
        plt.show()
else:
    print("Not found path")
    

**Permutation function**
Time complex: O(n!)

In [ ]:
res = []
def permutation(raw_list, prefix_list):
    if len(raw_list) == 0:
        res.append(prefix_list)
        return
    for i in range(len(raw_list)):
        post_list = raw_list[:i] + raw_list[i+1:]
        permutation(post_list, prefix_list + [raw_list[i]])

In [ ]:
pref_list = []
A = ['a', 'b', 'c', 'd', 'e', 'f']
permutation(A, pref_list)